# Submitting RayJobs against an existing RayCluster

In this notebook, we will go through the basics of using the SDK to:
 * Spin up a Ray cluster with our desired resources
 * Verify the status of this cluster
 * Submit a RayJob against that cluster
 * Verify the status of this job

## Creating the RayCluster

First, we'll need to import the relevant CodeFlare SDK packages. You can do this by executing the below cell.

In [ ]:
from codeflare_sdk import Cluster, ClusterConfiguration, RayJob

Run the below `oc login` command using your Token and Server URL. Ensure the command is prepended by `!` and not `%`. This will work when running both locally and within RHOAI.

In [ ]:
!oc login --token=<your-token> --server=<your-server-url>

Next we'll need to initalize our RayCluster and apply it. You can do this be executing the below cell.

In [ ]:
cluster = Cluster(ClusterConfiguration(
    name='rayjob-cluster',
    head_cpu_requests=1,
    head_cpu_limits=1,
    head_memory_requests=6,
    head_memory_limits=8,
    head_extended_resource_requests={'nvidia.com/gpu':1},
    worker_extended_resource_requests={'nvidia.com/gpu':1},
    num_workers=2,
    worker_cpu_requests='250m',
    worker_cpu_limits=1,
    worker_memory_requests=4,
    worker_memory_limits=6,

))

cluster.apply()

We can check the status of our cluster by executing the below cell. If it's not up immediately, run the cell a few more times until you see that it's in a 'running' state.

In [ ]:
cluster.status()

## Submitting the RayJob

Now we can create the RayJob that we want to submit against the running cluster. The process is quite similar to how we initialize and apply the cluster. 
In this context, we need to use the `cluster_name` variable to point it to our existing cluster.

For the sake of demonstration, the job we'll submit via the `entrypoint` is a single python command. In standard practice this would be pointed to a python training script.

We'll then call the `submit()` function to run the job against our cluster.

You can run the below cell to achieve this.

In [ ]:
rayjob = RayJob(
    job_name="sdk-test-job",
    cluster_name="rayjob-cluster",
    namespace="your-namespace",
    entrypoint="python -c 'import time; time.sleep(20)'",
)

rayjob.submit()

We can observe the status of the RayJob in the same way as the RayCluster by invoking the `status()` function via the below cell.

In [ ]:
rayjob.status()

This function will output different tables based on the RayJob's current status. You can re-run the cell multiple times to observe the changes as you need to. Once you've observed that the job has been completed, you can shut down the cluster we created earlier by executing the below cell.

In [ ]:
cluster.down()